In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import split,col,filter,isnan,when
from pyspark.sql.types import StringType,IntegerType
import pandas as pd
import matplotlib.pyplot as plt
from petastorm import make_reader
from petastorm.unischema import Unischema,UnischemaField
from petastorm.etl.dataset_metadata import materialize_dataset
from petastorm.codecs import ScalarCodec,NdarrayCodec
from petastorm.predicates import in_pseudorandom_split
from petastorm.transform import TransformSpec
import numpy as np
import os
import sys
import wfdb
import matplotlib.pyplot as plt
sys.path.append(os.path.join(os.getcwd(), ".."))
from operations.dataset_manager import Physionet_reader

In [ ]:
name = "mit-bih-noise-stress-test-database-1.0.0"
Physionet_reader(name)

## Trial : How to read physionet dataset


### Set path

In [ ]:
path_dataset = "/workspaces/ecg_evaluation/data/mit-bih-noise-stress-test-database-1.0.0"
if (not os.path.exists(path_dataset)):
    os.makedirs(path_dataset)

### How physionet Dataset are organized

Physionet dataset have the following format:
- __.dat__ files : Binary files containing amples of digitized signals. They store the waveformes but have no header
- __.hea__ files : Text file containing description of the content associated with the signals.
- __.atr__ files : Binary files containing the labels for a dataset

#### Example

In [ ]:
data = wfdb.rdsamp("/workspaces/ecg_evaluation/data/mit-bih-noise-stress-test-database-1.0.0/118e_6")
print(data[1])
real_data = data[0][:,0]
plt.plot(real_data)
plt.xlim([0,3600])


### Data sample at 360 Hz. Need resampling to 500 Hz

In [ ]:
## number of signal
nb_signal = data[1]["n_sig"]
## Desire time
time_window = 10
##Original sampling frequency
fs_ori = data[1]['fs']
##new sampling frequency 
fs_new = 500
##new length signal
N_old = data[1]['sig_len']
N_new = fs_new*time_window
## Calculate duration of the total signal
time_tot = N_old/fs_ori


## First, we have to resample the signal into 500 Hz.
scale  = fs_new/fs_ori
##new sample length
N_res = int(N_old*(fs_new/fs_ori))
resample_data = np.zeros([N_res,nb_signal])

for sig in range(nb_signal):
    sig_to_resamp = data[0][:,sig]
    resample_data[:,sig] = np.interp(np.linspace(0,time_tot,N_res,endpoint = False),
                                   np.linspace(0,time_tot,N_old,endpoint = False),
                                   sig_to_resamp)

print(resample_data.shape)
## Number of chunks : 
n = int(N_res/N_new)
## Array containing all the new sample (shape : [nb_signal,nb_chunk,N_new])
new_dataset = np.zeros([nb_signal,n,N_new])
for s in range(nb_signal):
    sig_study = resample_data[:,s]
    for chunks in range(n):
        new_dataset[s,chunks,:] = sig_study[N_new*chunks:N_new*(chunks+1)]

print(new_dataset.shape)



